<a href="https://colab.research.google.com/github/lrssv/TimeSeriesForecastingCOVID19/blob/master/INMET_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Multi-step ARIMA for data INMET Forecast using AUTOARIMA


In [1]:
#imports
import warnings
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import timedelta
import itertools
from statsmodels.tsa.arima_model import ARIMA
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#setting parameters for graphics
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 16,9

## Data treatment

In [41]:
#reading file
url = '/content/drive/My Drive/Data Files/inmet_2018-2019.txt'
read_table = pd.read_table(url, delim_whitespace=True, usecols=["ID",'ANO','MES','DIA','HORA',"RAD","VENTOVEL"])
df = pd.DataFrame(read_table)

In [42]:
#replacing negative radiation values
df = df.assign(RAD = df.RAD.where(df.RAD.ge(0)))
df.fillna(0)

,ID,ANO,MES,DIA,HORA,VENTOVEL,RAD
0,A521,2018,1,1,0,0.7,0.000
1,A521,2018,1,1,1,0.8,0.000
2,A521,2018,1,1,2,0.6,0.000
3,A521,2018,1,1,3,0.3,0.000
4,A521,2018,1,1,4,0.3,0.000
...,...,...,...,...,...,...,...
17493,A521,2019,12,31,19,1.9,1747.192
17494,A521,2019,12,31,20,4.3,577.485
17495,A521,2019,12,31,21,3.1,655.452
17496,A521,2019,12,31,22,2.8,43.999


In [45]:
#grouping the date + time
source_col_loc = df.columns.get_loc('ID') 

df['datetime'] = df.iloc[:,source_col_loc+1:source_col_loc+5].apply(
    lambda x: "-".join(x.astype(str)), axis=1)

data = df[['datetime','VENTOVEL']]

In [46]:
#mounting the datetime 
dataFormatada = pd.to_datetime(data['datetime'], format='%Y-%m-%d-%H')

d = {'date':dataFormatada, 'ventovel': df['VENTOVEL']}
f= {'date': dataFormatada}

dataFrame = pd.DataFrame(data=d) #datetime + ventovel
frameList = pd.DataFrame(data=f) #only datetime

frameList

,date
0,2018-01-01 00:00:00
1,2018-01-01 01:00:00
2,2018-01-01 02:00:00
3,2018-01-01 03:00:00
4,2018-01-01 04:00:00
...,...
17493,2019-12-31 19:00:00
17494,2019-12-31 20:00:00
17495,2019-12-31 21:00:00
17496,2019-12-31 22:00:00


In [47]:
#creating all dates 
#date_list = [pd.to_datetime('2018-01-01 01:00:00') + timedelta(hours=x) for x in range(date.shape[0])] (RASCUNHO)

serieStart = '2018-01-01 00:00:00' 
serieEnd = '2019-12-31 23:00:00'

date = pd.date_range(start=serieStart, end=serieEnd, freq='1H')

dt = {'date': date}
frameDate = pd.DataFrame(dt)

frameDate

,date
0,2018-01-01 00:00:00
1,2018-01-01 01:00:00
2,2018-01-01 02:00:00
3,2018-01-01 03:00:00
4,2018-01-01 04:00:00
...,...
17515,2019-12-31 19:00:00
17516,2019-12-31 20:00:00
17517,2019-12-31 21:00:00
17518,2019-12-31 22:00:00


In [ ]:
#TO DO:
#Inserir dados faltantes
# 1 - fazer um for que percorre todas as datas de 2018-2019
# 2 - if a data criada existe no banco de dados?
# 3 - se sim: faz nada 
# 4 - se não: inclui uma linha no banco de dados com a data e NaN para a velocidade do vento

#Substituir valores negativos por zero
#how to do?

In [ ]:
#including the missing dates 
#RASCUNHO
ventovel = df['VENTOVEL']
new_dates = []
new_values = []

for i in frameList:
  try:
    if frameList[i] == frameDate[i]:
      new_dates.append[frameList[i]]
      new_values.append[ventovel[i]]
  
  except:
    new_dates.append[frameDate[i]]
    new_values.append[np.nan]

In [ ]:
#interpolate NaN values
frameInterpolated = frameList.interpolate(method='polynomial', order=2)

In [ ]:
#transforming data in series
#series = pd.Series(frameInterpolated)

## AUTOARIMA implementation

In [ ]:
pip install pmdarima

In [ ]:
import pmdarima as pm

In [ ]:
auto = pm.auto_arima(series, d=n_diffs, seasonal=False, stepwise=True,
                     suppress_warnings=True, error_action="ignore", max_p=6,
                     max_order=None, trace=True)

In [ ]:
print(auto.order)

(1, 1, 2)


## ARIMA implementation

In [ ]:
import warnings
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import timedelta
import itertools
from statsmodels.tsa.arima_model import ARIMA
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [ ]:
def run_tests(series,cold_start=30,steps_ahead=1,configurations=[(1,1,1)]):

  begin = series.index.min() + timedelta(days=0)
  # List of all dates from begin + cold_start to end
  date_list = [begin + timedelta(days=cold_start) + timedelta(days=x) for x in range(len(series)-steps_ahead-cold_start)]


  results = []
  ex_count = 0

  for c in configurations:
    for date in tqdm(date_list):
      # Select data
      train = series[begin:date] 

      f_date = date + timedelta(days=steps_ahead) # The day we want the forecast for

      # Fit ARIMA
      try: # Try to use ARIMA
        mod = sm.tsa.statespace.SARIMAX(train, order=c) # Selected the parameters randomly
        res = mod.fit(disp=False)
        # Get forecast
        forecast = res.forecast(days=steps_ahead)[0] 
        results.append([f_date,forecast,series[f_date],np.abs(series[f_date]-res.forecast()[0]),'arima',c])
      except: # Use persistence
        results.append([f_date,series[date],series[f_date],np.abs(series[f_date]-series[date]),'persistence',None])
        ex_count = ex_count+1


  print('ARIMA ERRORS: ' + str(ex_count))

  return results

In [ ]:
def plot_results(series,forecasts,steps_ahead):

  plt.rcParams.update({'font.size': 15})
  figure(num=None, figsize=(15, 8), dpi=80, facecolor='w', edgecolor='k')

 

  plt.plot(dates,error,'--')
  plt.plot(series,'-')
  plt.plot(dates,forecasts,'.')
  plt.plot(dates,persist,'-')
  
  plt.legend(['Error','Observed','Forecast','Persist'])


  print('MAE: {} ESTD: {}'.format(np.mean(error),np.std(error)))

In [ ]:
cold_start = 30 
configurations = [(1,1,2)]

In [ ]:
results_one_day_ahead = run_tests(series,cold_start=cold_start,steps_ahead=1,configurations=configurations)

  0%|          | 0/17489 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
  0%|          | 1/17489 [00:00<1:34:43,  3.08it/s]/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
  0%|          | 2/17489 [00:00<1:36:52,  3.01it/s]/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
  0%|          | 3/17489 [00:01<1:37:45,  2.98it/s]/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
  0%|          | 4/17489 [00:01<1:

In [ ]:
plot_results(series,results_one_day_ahead,1)

In [ ]:
cold_start=30
steps_ahead=1

begin = series.index.min() + timedelta(days=0)
  # List of all dates from begin + cold_start to end
date_list = [begin + timedelta(days=cold_start) + timedelta(days=x) for x in range(len(series)-steps_ahead-cold_start)]



date_list


[Timestamp('2018-01-31 00:00:00'),
 Timestamp('2018-02-01 00:00:00'),
 Timestamp('2018-02-02 00:00:00'),
 Timestamp('2018-02-03 00:00:00'),
 Timestamp('2018-02-04 00:00:00'),
 Timestamp('2018-02-05 00:00:00'),
 Timestamp('2018-02-06 00:00:00'),
 Timestamp('2018-02-07 00:00:00'),
 Timestamp('2018-02-08 00:00:00'),
 Timestamp('2018-02-09 00:00:00'),
 Timestamp('2018-02-10 00:00:00'),
 Timestamp('2018-02-11 00:00:00'),
 Timestamp('2018-02-12 00:00:00'),
 Timestamp('2018-02-13 00:00:00'),
 Timestamp('2018-02-14 00:00:00'),
 Timestamp('2018-02-15 00:00:00'),
 Timestamp('2018-02-16 00:00:00'),
 Timestamp('2018-02-17 00:00:00'),
 Timestamp('2018-02-18 00:00:00'),
 Timestamp('2018-02-19 00:00:00'),
 Timestamp('2018-02-20 00:00:00'),
 Timestamp('2018-02-21 00:00:00'),
 Timestamp('2018-02-22 00:00:00'),
 Timestamp('2018-02-23 00:00:00'),
 Timestamp('2018-02-24 00:00:00'),
 Timestamp('2018-02-25 00:00:00'),
 Timestamp('2018-02-26 00:00:00'),
 Timestamp('2018-02-27 00:00:00'),
 Timestamp('2018-02-